In [ ]:
from numpy import *
import astropy.units as u
from astropy.constants import G, M_jup, R_jup, M_earth, R_earth, L_sun, M_sun, R_sun
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import extinction as ex
import scipy.stats as st

In [ ]:
db = pd.read_csv('recalculations.csv')
tracers = ['Ha','PaB','BrG']

## Usable objects:
Count the number of objects we can re-estimate and those we cannot. For those we cannot, tabulate how many are missing line flux as opposed to mass or radius.

Optional: print index and source name of each object as well as what parameters it is missing.

In [ ]:
total = 0
no_r = 0
no_m = 0
no_f = 0
for i in db.index:
    
    #for each parameter, set it to a boolean
    #if it is True, that parameter is NOT nan.
    
    d = not isnan(db.loc[i,'Plx Distance'])
    R = not isnan(db.loc[i,'Object Radius'])
    M = not isnan(db.loc[i,'Object Mass, Original'])
    flux = False
    for t in tracers:
        if not isnan(db.loc[i,t + ' Line Flux']):
            flux = True
    
    if (d and R and M and flux):
        #print(i,db.loc[i,'Source'],': good')
        total+=1
    elif (not d and R and M and flux):
        #print(i,db.loc[i,'Source'],': just distance')
        total+=1
    elif (not flux):
        #print(i,db.loc[i,'Source'],': NO FLUX')
        no_f+=1
    elif (R and not M):
        #print(i,db.loc[i,'Source'],': no mass')
        no_m+=1
    elif (not R and M):
        #print(i,db.loc[i,'Source'],': no radius')
        no_r+=1
    
print('Total usable:', total)
print('Need radius:',no_r)
print('Need mass:',no_m)
print('No flux:',no_f)

## Discrepant values for the same object:

Print objects that appear in the database multiple times with different mass estimates.

Print their names, indices, the mass difference in dex, and their references.

In [ ]:
#collect the mass used into one list: some are re-estimated while some are original
mass_used = []

for i in db.index:
    M = db.loc[i,'New Mass']
    if isnan(M):
        M = db.loc[i,'Object Mass, Original']    
    mass_used.append(M)

db['Adopted Mass'] = mass_used

In [ ]:
#sort database by source name. Make a list of masses used.
sourceSorted = db.sort_values(by='Source')
sm = log10(sourceSorted['Adopted Mass'].tolist())

In [ ]:
#Iterate through sorted database.
#If a source appears more than once, check whether the adopted masses are identical.
#If not, print relevant information.

#Set how much of a difference in mass (in dex) you want to flag.
mass_diff_limit = 0.25

for i in range(len(sourceSorted)-1):
    
    #check if source appears more than once
    if sourceSorted.loc[sourceSorted.index[i],'Source']==sourceSorted.loc[sourceSorted.index[i+1],'Source']:
        
        #check if adopted masses are identical
        if abs(sm[i]-sm[i+1]) > mass_diff_limit:
            
            #print relevant information:
            #source name and the pair of indices
            #the two masses
            #the difference between the masses
            #the paper references for the objects
            print(sourceSorted.loc[sourceSorted.index[i],'Source'], ':\n Indices:',
                  sourceSorted.index[i],sourceSorted.index[i+1],
                  '\n Masses: ', round(10**sm[i],3),'and',round(10**sm[i+1],3),
                  '\n Difference:', round(sm[i]-sm[i+1],2), 'dex',
                  '\n References:',sourceSorted.loc[sourceSorted.index[i],'Reference'],
                  sourceSorted.loc[sourceSorted.index[i+1],'Reference'],'\n')